    !python -m pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html
    !python -m pip install skorch

In [1]:
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

In [2]:
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cpu')

In [6]:
import os
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
import torch.nn.functional as F

from skorch import NeuralNetClassifier
from skorch.callbacks import (Checkpoint,
                              TrainEndCheckpoint,
                              LoadInitState)

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split

In [4]:
from  pickle import load, dump

________

In [8]:
x, y = make_classification(
    n_samples=100_000,
    n_features=20,
    n_informative=8,
    n_classes=2,
    n_clusters_per_class=1,
    weights=[0.5],
    shuffle=True,
    random_state=1)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.01)

___

In [9]:
cp = Checkpoint(dirname='pytorch/checkpoint')
train_end_cp = TrainEndCheckpoint(dirname='pytorch/checkpoint')
load_state = LoadInitState(cp)

In [7]:
class MyModule(nn.Module):
    def __init__(self, num_units=16, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.output = nn.Linear(num_units, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
       #X = self.sigmoid(self.output(X))
        X = F.softmax(self.output(X), dim=1)
        return X

In [9]:
net = NeuralNetBinaryClassifier(
    MyModule,
    device='cuda',
    max_epochs=15,
    lr=0.01,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    callbacks=[cp, train_end_cp]
)

In [9]:
net.__dict__

{'module': __main__.MyModule,
 'criterion': torch.nn.modules.loss.NLLLoss,
 'optimizer': torch.optim.sgd.SGD,
 'lr': 0.01,
 'max_epochs': 15,
 'batch_size': 128,
 'iterator_train': torch.utils.data.dataloader.DataLoader,
 'iterator_valid': torch.utils.data.dataloader.DataLoader,
 'dataset': skorch.dataset.Dataset,
 'train_split': <skorch.dataset.CVSplit object at 0x7f78a26e4810>,
 'callbacks': [<skorch.callbacks.training.Checkpoint at 0x7f78a21fa7d0>,
 'warm_start': False,
 'verbose': 1,
 'device': 'cuda',
 'iterator_train__shuffle': True,
 'history_': None,
 'initialized_': False,
 'virtual_params_': {},
 'classes': None}

In [10]:
net.fit(X, y);

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.2860       0.9564        0.1422     +  0.9235
      2        0.1824       0.9615        0.1225     +  0.8990
      3        0.1623       0.9643        0.1154     +  0.9918
      4        0.1523       0.9668        0.1107     +  1.1005
      5        0.1457       0.9682        0.1072     +  1.0168
      6        0.1427       0.9699        0.1043     +  0.9786
      7        0.1369       0.9713        0.1019     +  0.9113
      8        0.1330       0.9728        0.0997     +  0.8996
      9        0.1316       0.9744        0.0978     +  0.9434
     10        0.1300       0.9751        0.0965     +  0.9379
     11        0.1274       0.9756        0.0954     +  0.9181
     12        0.1260       0.9764        0.0943     +  0.9204
     13        0.1252       0.9768        0.0934     +  0.9210
     14        0.1249       0.9772        0.0924     + 

In [11]:
print("Model's state_dict:")
for param_tensor in net.__dict__['module_'].state_dict():
    print(param_tensor, "\t", net.__dict__['module_'].state_dict()[param_tensor].size())

Model's state_dict:
dense0.weight 	 torch.Size([16, 20])
dense0.bias 	 torch.Size([16])
output.weight 	 torch.Size([2, 16])
output.bias 	 torch.Size([2])


In [12]:
# print("Model's parameters:")
# for param_tensor in net.__dict__['module_'].parameters():
#     print(param_tensor, "\t", net.__dict__['module_'].parameters())

In [13]:
X, y = make_classification(
    n_samples=10,
    n_features=20,
    n_informative=5,
    n_classes=2,
    n_clusters_per_class=1,
    weights=[0.5],
    shuffle=True,
    random_state=1)

X = X.astype(np.float32)
y = y.astype(np.int64)

y_proba = net.predict_proba(X)
np.argmax(y_proba, axis=1)

array([1, 0, 1, 1, 1, 0, 1, 0, 0, 0])

In [14]:
y

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 0])

In [15]:
# new_net = NeuralNetClassifier(
#     MyModule,
#     device='cpu',
#     max_epochs=15,
#     lr=0.01,
#     # Shuffle training data on each epoch
#     iterator_train__shuffle=True,
#     #callbacks=[cp, train_end_cp]
# )

# new_net.initialize()
# new_net.load_params(checkpoint=train_end_cp)

# X, y = make_classification(
#     n_samples=10,
#     n_features=20,
#     n_informative=5,
#     n_classes=2,
#     n_clusters_per_class=1,
#     weights=[0.5],
#     shuffle=True,
#     random_state=1)

# X = X.astype(np.float32)
# y = y.astype(np.int64)

# y_pred = new_net.predict(X)
# y_pred

In [16]:
net.save_params(f_params='pytorch/save-params-fn.pkl')

In [50]:
os.stat('pytorch/save-params-fn.pkl').st_size

2365

In [17]:
new_net = None

In [18]:
new_net = NeuralNetClassifier(
    MyModule,
    max_epochs=3,
    lr=0.001,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
    # callbacks=[cp, train_end_cp, load_state]
)

In [19]:
new_net.initialize()
new_net.load_params(f_params='pytorch/save-params-fn.pkl')

In [20]:
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=5,
    n_classes=2,
    n_clusters_per_class=1,
    weights=[0.5],
    shuffle=True,
    random_state=1)

X = X.astype(np.float32)
y = y.astype(np.int64)

In [21]:
new_net.fit(X, y);

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6499       0.6250        0.6280  0.0091
      2        0.6619       0.6250        0.6266  0.0103
      3        0.6577       0.6300        0.6252  0.0101


In [22]:
X, y = make_classification(
    n_samples=10,
    n_features=20,
    n_informative=5,
    n_classes=2,
    n_clusters_per_class=1,
    weights=[0.5],
    shuffle=True,
    random_state=1)

X = X.astype(np.float32)
y = y.astype(np.int64)


In [23]:
y_proba = new_net.predict_proba(X)
np.argmax(y_proba, axis=1)

array([1, 0, 0, 1, 0, 1, 1, 0, 1, 1])

In [24]:
y

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 0])

____

In [25]:
# Inference model doesn't need original class, only pickle

In [37]:
loaded_model = None

In [38]:
dump(new_net, open('pytorch/dump.pkl', 'wb'))

In [51]:
os.stat('pytorch/dump.pkl').st_size

7277

In [39]:
loaded_model = load(open('pytorch/dump.pkl', 'rb'))

In [40]:
y_proba = loaded_model.predict_proba(X)
np.argmax(y_proba, axis=1)

array([1, 0, 0, 1, 0, 1, 1, 0, 1, 1])

____

In [41]:
### pytorch in a sklearn pipeline

In [42]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', loaded_model),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6922       0.5000        0.8184  0.0011
      2        0.7429       0.5000        0.8182  0.0013
      3        0.7381       0.5000        0.8181  0.0011


In [43]:
np.argmax(y_proba, axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
y

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 0])

____________

In [45]:
dump(pipe, open('pytorch/pipeline.pkl', 'wb'))

In [53]:
os.stat('pytorch/pipeline.pkl').st_size

7767

In [46]:
loaded_pipe = load(open('pytorch/pipeline.pkl', 'rb'))
pipe

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('net',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (dense0): Linear(in_features=20, out_features=16, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (output): Linear(in_features=16, out_features=2, bias=True)
    (sigmoid): Sigmoid()
  ),
))],
         verbose=False)

In [36]:
pipe.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])